<a href="https://colab.research.google.com/github/KalpanaAK/Codsoft_intern_tasks/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab setup - run this first
!pip install -q transformers sentence-transformers torch torchvision torchaudio
!pip install -q nltk spacy textstat python-docx language-tool-python
!python -m spacy download en_core_web_sm

# Optional but useful: install faster tokenizers
!pip install -q sentencepiece

print("Install complete. Runtime may need restart for some libraries (usually not required).")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 107.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Install complete. Runtime may need restart for some libraries (usually not required).


In [4]:
import os
import re
import math
from typing import List, Tuple
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# FIX: Remove old Java and install Java 17 for language_tool_python
!apt-get purge openjdk-11* -y > /dev/null
!apt-get autoremove -y > /dev/null
!apt-get install -y openjdk-17-jre-headless > /dev/null

import spacy
nlp_spacy = spacy.load("en_core_web_sm")

from sentence_transformers import SentenceTransformer, util
from transformers import T5ForConditionalGeneration, T5TokenizerFast

import language_tool_python
import textstat
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

# lazy-load heavy models variables (will be loaded on demand)
SBERT_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
T5_MODEL_NAME = "t5-small"

sbert_model = None
t5_tokenizer = None
t5_model = None

# initialize LanguageTool
lang_tool = language_tool_python.LanguageTool('en-US')

print("✅ LanguageTool initialized with Java 17 successfully!")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
INFO:language_tool_python.download_lt:Unzipping /tmp/tmpn88r4935.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://internal1.languagetool.org/snapshots/LanguageTool-latest-snapshot.zip to /root/.cache/language_tool_python.


✅ LanguageTool initialized with Java 17 successfully!


In [5]:
def load_models(load_sbert=True, load_t5=True):
    global sbert_model, t5_tokenizer, t5_model
    if load_sbert and sbert_model is None:
        print("Loading SBERT model...")
        sbert_model = SentenceTransformer(SBERT_MODEL_NAME)
        print("SBERT loaded.")
    if load_t5 and t5_model is None:
        print("Loading T5 model & tokenizer...")
        t5_tokenizer = T5TokenizerFast.from_pretrained(T5_MODEL_NAME)
        t5_model = T5ForConditionalGeneration.from_pretrained(T5_MODEL_NAME)
        print("T5 loaded.")


In [6]:
# Replace the text below with your own project idea / keywords
user_input_text = """AI-based intrusion detection for cloud networks using lightweight ML models and federated learning for privacy-preserving detection."""

print("User input ready:")
print(user_input_text)


User input ready:
AI-based intrusion detection for cloud networks using lightweight ML models and federated learning for privacy-preserving detection.


In [9]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download ALL required NLTK resources (new & old names)
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text: str) -> str:
    text = re.sub(r'\s+', ' ', text.strip())  # remove extra spaces

    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Remove non-alphanumeric & stopwords
    tokens_clean = [t for t in tokens if t.isalnum() and t.lower() not in stop_words]

    # POS Tagging (handles both new & old tagger names)
    try:
        pos_tags = nltk.pos_tag(tokens_clean, tagset='universal')
    except:
        pos_tags = nltk.pos_tag(tokens_clean)

    # Lemmatization
    lemmas = [lemmatizer.lemmatize(word.lower()) for word, pos in pos_tags]

    return " ".join(lemmas)

preprocessed = preprocess_text(user_input_text)
print("✅ Preprocessed text:", preprocessed)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


✅ Preprocessed text: intrusion detection cloud network using lightweight ml model federated learning detection


In [10]:
# extract noun phrases and keywords
def extract_keywords_spacy(text: str, top_k=8) -> List[str]:
    doc = nlp_spacy(text)
    # noun chunks
    noun_chunks = [chunk.text for chunk in doc.noun_chunks]
    # single nouns & proper nouns
    nouns = [token.text for token in doc if token.pos_ in ("NOUN", "PROPN")]
    candidates = noun_chunks + nouns
    # simple dedupe and rank by frequency
    freq = {}
    for c in candidates:
        key = c.lower()
        freq[key] = freq.get(key, 0) + 1
    sorted_kws = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    return [k for k, _ in sorted_kws][:top_k]

keywords = extract_keywords_spacy(user_input_text)
print("Keywords / Concepts:", keywords)


Keywords / Concepts: ['detection', 'ai-based intrusion detection', 'cloud networks', 'lightweight ml models', 'federated learning', 'privacy-preserving detection', 'ai', 'intrusion']


In [11]:
# Template-based generator (fast)
def generate_titles_templates(keywords: List[str]) -> List[str]:
    templates = [
        "{0} for {1}: A {2} Approach",
        "An Efficient {0} Using {1} for {2}",
        "{0}-based {1} for {2}",
        "Privacy-Preserving {0} for {1}",
        "Lightweight {0} with {1} in {2}"
    ]
    titles = []
    # choose first three keywords safely
    main = keywords[0] if len(keywords) > 0 else "System"
    sec = keywords[1] if len(keywords) > 1 else "Approach"
    third = keywords[2] if len(keywords) > 2 else "Application"
    for tpl in templates:
        titles.append(tpl.format(main.title(), sec.title(), third.title()))
    return titles

# Model-based generation using T5 (prompting)
def generate_titles_t5(prompt: str, n=5):
    load_models(load_sbert=False, load_t5=True)
    input_text = "generate titles: " + prompt
    inputs = t5_tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=128)
    outputs = t5_model.generate(inputs, max_length=40, num_return_sequences=n, do_sample=True, top_k=50, top_p=0.95)
    titles = [t5_tokenizer.decode(o, skip_special_tokens=True) for o in outputs]
    return titles

# Semantic Ranking for title candidates using SBERT cosine similarity to original idea
def rank_titles_by_semantic_similarity(candidates: List[str], source: str) -> List[Tuple[str, float]]:
    load_models(load_sbert=True, load_t5=False)
    emb_source = sbert_model.encode(source, convert_to_tensor=True)
    emb_cands = sbert_model.encode(candidates, convert_to_tensor=True)
    scores = util.cos_sim(emb_cands, emb_source).cpu().numpy().flatten().tolist()
    ranked = sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)
    return ranked

# Generate & rank
template_titles = generate_titles_templates(keywords)
print("Template titles:")
for t in template_titles: print(" -", t)

# (Optional) T5 titles - uncomment to use (can be slow)
# t5_titles = generate_titles_t5(user_input_text, n=5)
# print("T5-generated titles:", t5_titles)

ranked_templates = rank_titles_by_semantic_similarity(template_titles, user_input_text)
print("\nRanked template titles (score):")
for t, s in ranked_templates:
    print(f"{s:.4f}  -  {t}")


Template titles:
 - Detection for Ai-Based Intrusion Detection: A Cloud Networks Approach
 - An Efficient Detection Using Ai-Based Intrusion Detection for Cloud Networks
 - Detection-based Ai-Based Intrusion Detection for Cloud Networks
 - Privacy-Preserving Detection for Ai-Based Intrusion Detection
 - Lightweight Detection with Ai-Based Intrusion Detection in Cloud Networks
Loading SBERT model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SBERT loaded.

Ranked template titles (score):
0.6933  -  Privacy-Preserving Detection for Ai-Based Intrusion Detection
0.6910  -  Detection for Ai-Based Intrusion Detection: A Cloud Networks Approach
0.6799  -  An Efficient Detection Using Ai-Based Intrusion Detection for Cloud Networks
0.6770  -  Lightweight Detection with Ai-Based Intrusion Detection in Cloud Networks
0.6714  -  Detection-based Ai-Based Intrusion Detection for Cloud Networks


In [12]:
# Generate abstract using T5 by prompting about the idea
def generate_abstract_t5(idea: str, max_len=200):
    load_models(load_sbert=False, load_t5=True)
    prompt = "summarize: " + idea
    inputs = t5_tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=256)
    outputs = t5_model.generate(inputs, max_length= max_len, num_beams=4, early_stopping=True)
    abstract = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return abstract

# Template-based abstract builder (outline -> fill)
def generate_abstract_template(idea: str, keywords: List[str]):
    # Simple structured abstract builder
    problem = f"This work addresses {keywords[0]} and related challenges in {keywords[1]}." if len(keywords)>1 else f"This work addresses {idea}."
    aim = f"The aim is to design an efficient approach for {keywords[0]}."
    method = "We propose a hybrid pipeline combining semantic embeddings, lightweight machine learning models, and privacy-aware training (e.g., federated learning)."
    result = "Expected outcomes include improved detection accuracy with reduced communication overhead."
    application = "This approach is applicable to cloud-based intrusion detection systems and resource-constrained environments."
    abstract = " ".join([problem, aim, method, result, application])
    return abstract

# Generate both versions
abstract_template = generate_abstract_template(user_input_text, keywords)
print("Template Abstract:\n", abstract_template[:800])

# (Optional) T5 abstract (uncomment to use)
# t5_abstract = generate_abstract_t5(user_input_text, max_len=200)
# print("\nT5 Abstract:\n", t5_abstract)


Template Abstract:
 This work addresses detection and related challenges in ai-based intrusion detection. The aim is to design an efficient approach for detection. We propose a hybrid pipeline combining semantic embeddings, lightweight machine learning models, and privacy-aware training (e.g., federated learning). Expected outcomes include improved detection accuracy with reduced communication overhead. This approach is applicable to cloud-based intrusion detection systems and resource-constrained environments.


In [13]:
# Grammar correction using LanguageTool
def grammar_correct(text: str) -> str:
    matches = lang_tool.check(text)
    corrected = language_tool_python.utils.correct(text, matches)
    return corrected

# Coherence scoring (readability + simple semantic fit)
def coherence_score(text: str, source: str) -> float:
    # Combine readability & semantic similarity
    r = textstat.flesch_reading_ease(text)
    load_models(load_sbert=True, load_t5=False)
    sem = util.cos_sim(sbert_model.encode(text, convert_to_tensor=True),
                       sbert_model.encode(source, convert_to_tensor=True)).item()
    # Normalize r (~0-100) and combined score
    r_norm = max(min(r, 100), 0) / 100.0
    sem_norm = (sem + 1) / 2  # cosine similarity [-1,1] -> [0,1]
    score = 0.5*r_norm + 0.5*sem_norm
    return score

# Simple "plagiarism-like" check: compute similarity to candidate titles / previously provided text
def semantic_similarity(a: str, b: str) -> float:
    load_models(load_sbert=True, load_t5=False)
    return util.cos_sim(sbert_model.encode(a, convert_to_tensor=True),
                        sbert_model.encode(b, convert_to_tensor=True)).item()

# Run enhancer on the template abstract
abstract_enhanced = grammar_correct(abstract_template)
score_before = coherence_score(abstract_template, user_input_text)
score_after = coherence_score(abstract_enhanced, user_input_text)
print("Before correction coherence:", round(score_before, 3))
print("After correction coherence:", round(score_after, 3))
print("\nEnhanced Abstract:\n", abstract_enhanced)


Before correction coherence: 0.505
After correction coherence: 0.505

Enhanced Abstract:
 This work addresses detection and related challenges in ai-based intrusion detection. The aim is to design an efficient approach for detection. We propose a hybrid pipeline combining semantic embeddings, lightweight machine learning models, and privacy-aware training (e.g., federated learning). Expected outcomes include improved detection accuracy with reduced communication overhead. This approach is applicable to cloud-based intrusion detection systems and resource-constrained environments.


In [14]:
# Paraphrase with T5 (optional)
def paraphrase_t5(text: str, num_return_sequences=3):
    load_models(load_sbert=False, load_t5=True)
    prefix = "paraphrase: "
    input_text = prefix + text
    encoding = t5_tokenizer.encode_plus(input_text, padding=True, return_tensors="pt", truncation=True)
    outputs = t5_model.generate(
        input_ids=encoding['input_ids'],
        attention_mask=encoding['attention_mask'],
        max_length=200,
        do_sample=True,
        top_k=120,
        top_p=0.98,
        num_return_sequences=num_return_sequences
    )
    return [t5_tokenizer.decode(out, skip_special_tokens=True) for out in outputs]

# Uncomment to paraphrase
# paras = paraphrase_t5(abstract_enhanced, num_return_sequences=2)
# print("Paraphrases:\n", paras)


In [15]:
def create_styled_doc(title: str, abstract: str, authors: List[str]=None, filename="generated_abstract.docx"):
    doc = Document()
    # Title
    p = doc.add_paragraph()
    run = p.add_run(title)
    run.bold = True
    run.font.name = 'Times New Roman'
    run.font.size = Pt(14)
    p.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Authors
    if authors:
        p2 = doc.add_paragraph()
        run2 = p2.add_run(", ".join(authors))
        run2.font.name = 'Times New Roman'
        run2.font.size = Pt(11)
        p2.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    doc.add_paragraph()  # spacing

    # Abstract heading
    p3 = doc.add_paragraph()
    run3 = p3.add_run("Abstract")
    run3.bold = True
    run3.font.name = 'Times New Roman'
    run3.font.size = Pt(12)
    p3.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

    # Abstract text
    abs_p = doc.add_paragraph()
    abs_run = abs_p.add_run(abstract)
    abs_run.font.name = 'Times New Roman'
    abs_run.font.size = Pt(12)
    abs_p.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY

    # Save
    doc.save(filename)
    return filename

# Create doc with the top ranked title
top_title = ranked_templates[0][0]
docfile = create_styled_doc(top_title, abstract_enhanced, authors=["Santhiya M", "Kalpana A"], filename="AI_Title_Abstract_Generated.docx")
print("DOCX saved to:", docfile)


DOCX saved to: AI_Title_Abstract_Generated.docx


In [16]:
from IPython.display import FileLink, display

print("Top-ranked Title:\n", top_title)
print("\nFinal Abstract:\n", abstract_enhanced)
print("\nCoherence score (final):", round(score_after, 3))
print("\nReadability (Flesch reading ease):", textstat.flesch_reading_ease(abstract_enhanced))
# Provide download link for the .docx
display(FileLink(docfile))


Top-ranked Title:
 Privacy-Preserving Detection for Ai-Based Intrusion Detection

Final Abstract:
 This work addresses detection and related challenges in ai-based intrusion detection. The aim is to design an efficient approach for detection. We propose a hybrid pipeline combining semantic embeddings, lightweight machine learning models, and privacy-aware training (e.g., federated learning). Expected outcomes include improved detection accuracy with reduced communication overhead. This approach is applicable to cloud-based intrusion detection systems and resource-constrained environments.

Coherence score (final): 0.505

Readability (Flesch reading ease): 11.3831475409836


/content/AI_Title_Abstract_Generated.docx

In [17]:
from IPython.display import FileLink, display
import pandas as pd

def show_full_output():
    print("\n" + "="*90)
    print("🧠 FINAL OUTPUT – AI Title & Abstract Generator")
    print("="*90 + "\n")

    # 1. Show preprocessed text
    print("🔹 Preprocessed Text:")
    print(preprocessed)
    print("\n" + "-"*90)

    # 2. Show keywords
    print("🔹 Extracted Keywords:")
    print(", ".join(keywords))
    print("\n" + "-"*90)

    # 3. Show all generated titles (ranked)
    print("🔹 Generated Titles (Ranked by Semantic Similarity):")
    df = pd.DataFrame(ranked_templates, columns=["Title", "Similarity Score"])
    df["Similarity Score"] = df["Similarity Score"].round(4)
    display(df)
    print("\n" + "-"*90)

    # 4. Show final enhanced abstract
    print("📝 Final Enhanced Abstract:\n")
    print(abstract_enhanced)
    print("\n" + "-"*90)

    # 5. Show evaluation
    print("📊 Evaluation Scores:")
    print(f"Coherence Score: {round(score_after, 3)}")
    print(f"Readability (Flesch Reading Ease): {textstat.flesch_reading_ease(abstract_enhanced)}")
    print("\n" + "-"*90)

    # 6. Show DOCX download link
    print("📁 Download Final Formatted Abstract Document:")
    display(FileLink(docfile))
    print("\n" + "="*90)

# Run final combined output
show_full_output()



🧠 FINAL OUTPUT – AI Title & Abstract Generator

🔹 Preprocessed Text:
intrusion detection cloud network using lightweight ml model federated learning detection

------------------------------------------------------------------------------------------
🔹 Extracted Keywords:
detection, ai-based intrusion detection, cloud networks, lightweight ml models, federated learning, privacy-preserving detection, ai, intrusion

------------------------------------------------------------------------------------------
🔹 Generated Titles (Ranked by Semantic Similarity):


,Title,Similarity Score
0,Privacy-Preserving Detection for Ai-Based Intr...,0.6933
1,Detection for Ai-Based Intrusion Detection: A ...,0.6910
2,An Efficient Detection Using Ai-Based Intrusio...,0.6799
3,Lightweight Detection with Ai-Based Intrusion ...,0.6770
4,Detection-based Ai-Based Intrusion Detection f...,0.6714



------------------------------------------------------------------------------------------
📝 Final Enhanced Abstract:

This work addresses detection and related challenges in ai-based intrusion detection. The aim is to design an efficient approach for detection. We propose a hybrid pipeline combining semantic embeddings, lightweight machine learning models, and privacy-aware training (e.g., federated learning). Expected outcomes include improved detection accuracy with reduced communication overhead. This approach is applicable to cloud-based intrusion detection systems and resource-constrained environments.

------------------------------------------------------------------------------------------
📊 Evaluation Scores:
Coherence Score: 0.505
Readability (Flesch Reading Ease): 11.3831475409836

------------------------------------------------------------------------------------------
📁 Download Final Formatted Abstract Document:


/content/AI_Title_Abstract_Generated.docx